In [3]:
import json
from pathlib import Path

import numpy as np
import pandas as pd

from sklearn.model_selection import KFold, learning_curve
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib
import matplotlib.pyplot as plt

DATA_PATH = "../data/gold/salaries_full.parquet"
ARTIFACTS_DIR = Path("../artifacts/mlp")

ARTIFACTS_DIR.mkdir(parents=True, exist_ok=True)

df = pd.read_parquet(DATA_PATH)

features_finais = [
    "experience_level_encoded",
    "company_size_encoded",
    "remote_ratio",
    "is_US",
    "job_type_Full-time",
    "job_type_Contract",
    "job_type_Part-time",
]

features_presentes = [c for c in features_finais if c in df.columns]

if "salary_in_usd" not in df.columns:
    raise ValueError("A coluna 'salary_in_usd' (target) não foi encontrada no dataset.")

X = df[features_presentes].copy()
y = df["salary_in_usd"].values

X = X.astype(float)

print("Features usadas:", features_presentes)
print("Formato de X:", X.shape)
print("Formato de y:", y.shape)

def create_model_pipeline():
    mlp = MLPRegressor(
        hidden_layer_sizes=(4, 8, 16),
        activation="relu",
        solver="adam",
        random_state=42,
        max_iter=1000,
        early_stopping=True,
        n_iter_no_change=20,
        validation_fraction=0.1,
    )
    pipeline = Pipeline(
        steps=[
            ("scaler", StandardScaler()),
            ("mlp", mlp),
        ]
    )
    return pipeline

K = 10
kf = KFold(n_splits=K, shuffle=True, random_state=42)

fold_metrics = []
best_fold_index = None
best_fold_mse = np.inf

for fold_idx, (train_idx, val_idx) in enumerate(kf.split(X), start=1):
    print(f"\n===== Fold {fold_idx}/{K} =====")
    
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]
    
    model = create_model_pipeline()
    
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_val)
    
    mse = mean_squared_error(y_val, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_val, y_pred)
    r2 = r2_score(y_val, y_pred)
    
    metrics = {
        "fold": fold_idx,
        "n_train": int(len(train_idx)),
        "n_val": int(len(val_idx)),
        "mse": float(mse),
        "rmse": float(rmse),
        "mae": float(mae),
        "r2": float(r2),
    }
    fold_metrics.append(metrics)
    
    print("Métricas do fold:")
    print(json.dumps(metrics, indent=4))
    
    fold_dir = ARTIFACTS_DIR / f"fold_{fold_idx}"
    fold_dir.mkdir(parents=True, exist_ok=True)
    
    joblib.dump(model, fold_dir / "model.pkl")
    
    with open(fold_dir / "metrics.json", "w") as f:
        json.dump(metrics, f, indent=4)
    
    if mse < best_fold_mse:
        best_fold_mse = mse
        best_fold_index = fold_idx

print("\n===== Resumo K-Fold =====")
for m in fold_metrics:
    print(
        f"Fold {m['fold']}: MSE={m['mse']:.2f}, RMSE={m['rmse']:.2f}, "
        f"MAE={m['mae']:.2f}, R2={m['r2']:.4f}"
    )

mse_values = [m["mse"] for m in fold_metrics]
rmse_values = [m["rmse"] for m in fold_metrics]
mae_values = [m["mae"] for m in fold_metrics]
r2_values = [m["r2"] for m in fold_metrics]
fold_ids = [m["fold"] for m in fold_metrics]

mean_mse = float(np.mean(mse_values))
mean_rmse = float(np.mean(rmse_values))
mean_mae = float(np.mean(mae_values))
mean_r2 = float(np.mean(r2_values))

summary_metrics = {
    "k": K,
    "mean_mse": mean_mse,
    "mean_rmse": mean_rmse,
    "mean_mae": mean_mae,
    "mean_r2": mean_r2,
    "best_fold": int(best_fold_index),
    "best_fold_mse": float(best_fold_mse),
}

print("\nMédias das métricas nos 10 folds:")
print(json.dumps(summary_metrics, indent=4))

with open(ARTIFACTS_DIR / "kfold_summary.json", "w") as f:
    json.dump(summary_metrics, f, indent=4)

plt.figure()
plt.plot(fold_ids, mse_values, marker="o")
plt.xlabel("Fold")
plt.ylabel("MSE")
plt.title("MSE por Fold")
plt.grid(True)
plt.savefig(ARTIFACTS_DIR / "mse_per_fold.png", bbox_inches="tight")
plt.close()

plt.figure()
plt.plot(fold_ids, rmse_values, marker="o")
plt.xlabel("Fold")
plt.ylabel("RMSE")
plt.title("RMSE por Fold")
plt.grid(True)
plt.savefig(ARTIFACTS_DIR / "rmse_per_fold.png", bbox_inches="tight")
plt.close()

plt.figure()
plt.plot(fold_ids, mae_values, marker="o")
plt.xlabel("Fold")
plt.ylabel("MAE")
plt.title("MAE por Fold")
plt.grid(True)
plt.savefig(ARTIFACTS_DIR / "mae_per_fold.png", bbox_inches="tight")
plt.close()

plt.figure()
plt.plot(fold_ids, r2_values, marker="o")
plt.xlabel("Fold")
plt.ylabel("R2")
plt.title("R2 por Fold")
plt.grid(True)
plt.savefig(ARTIFACTS_DIR / "r2_per_fold.png", bbox_inches="tight")
plt.close()


train_sizes, train_scores, val_scores = learning_curve(
    create_model_pipeline(),
    X,
    y,
    cv=KFold(n_splits=K, shuffle=True, random_state=42),
    scoring="r2",
    train_sizes=np.linspace(0.1, 1.0, 5),
    n_jobs=None,
)

train_scores_mean = np.mean(train_scores, axis=1)
val_scores_mean = np.mean(val_scores, axis=1)

plt.figure()
plt.plot(train_sizes, train_scores_mean, marker="o", label="Treino")
plt.plot(train_sizes, val_scores_mean, marker="o", label="Validação")
plt.xlabel("Tamanho do conjunto de treino")
plt.ylabel("R2")
plt.title("Curva de Aprendizado - MLP")
plt.legend()
plt.grid(True)
plt.savefig(ARTIFACTS_DIR / "learning_curve.png", bbox_inches="tight")
plt.close()


print("\n===== Treinando modelo final em full-data =====")

final_model = create_model_pipeline()
final_model.fit(X, y)

y_pred_full = final_model.predict(X)

final_mse = mean_squared_error(y, y_pred_full)
final_rmse = np.sqrt(final_mse)
final_mae = mean_absolute_error(y, y_pred_full)
final_r2 = r2_score(y, y_pred_full)

final_metrics = {
    "mse": float(final_mse),
    "rmse": float(final_rmse),
    "mae": float(final_mae),
    "r2": float(final_r2),
}

print("\n===== Métricas do Modelo Final (full-data) =====")
print(json.dumps(final_metrics, indent=4))

with open(ARTIFACTS_DIR / "final_model_metrics.json", "w") as f:
    json.dump(final_metrics, f, indent=4)

plt.figure()
plt.scatter(y, y_pred_full, alpha=0.5)
min_val = min(y.min(), y_pred_full.min())
max_val = max(y.max(), y_pred_full.max())
plt.plot([min_val, max_val], [min_val, max_val], linestyle="--")
plt.xlabel("Salário Real (USD)")
plt.ylabel("Salário Predito (USD)")
plt.title("Salário Real vs Predito - Modelo Final")
plt.savefig(ARTIFACTS_DIR / "final_pred_vs_true.png", bbox_inches="tight")
plt.close()

residuals = y - y_pred_full

plt.figure()
plt.scatter(y_pred_full, residuals, alpha=0.5)
plt.axhline(0, linestyle="--")
plt.xlabel("Salário Predito (USD)")
plt.ylabel("Resíduo (Real - Predito)")
plt.title("Resíduos vs Predito - Modelo Final")
plt.savefig(ARTIFACTS_DIR / "final_residuals_vs_pred.png", bbox_inches="tight")
plt.close()

joblib.dump(final_model, ARTIFACTS_DIR / "final_model.pkl")

print("Modelo final salvo em:", ARTIFACTS_DIR / "final_model.pkl")
print("Resumo das métricas de K-Fold salvo em:", ARTIFACTS_DIR / "kfold_summary.json")
print("Métricas do modelo final salvas em:", ARTIFACTS_DIR / "final_model_metrics.json")


Features usadas: ['experience_level_encoded', 'company_size_encoded', 'remote_ratio', 'is_US', 'job_type_Full-time', 'job_type_Contract', 'job_type_Part-time']
Formato de X: (22789, 7)
Formato de y: (22789,)

===== Fold 1/10 =====
Métricas do fold:
{
    "fold": 1,
    "n_train": 20510,
    "n_val": 2279,
    "mse": 1043699724.5125655,
    "rmse": 32306.341862126166,
    "mae": 26497.49942794129,
    "r2": 0.3647476715706841
}

===== Fold 2/10 =====
Métricas do fold:
{
    "fold": 2,
    "n_train": 20510,
    "n_val": 2279,
    "mse": 1048961253.4731559,
    "rmse": 32387.671319086156,
    "mae": 26731.75096361555,
    "r2": 0.3720895569314582
}

===== Fold 3/10 =====
Métricas do fold:
{
    "fold": 3,
    "n_train": 20510,
    "n_val": 2279,
    "mse": 1039770646.4696362,
    "rmse": 32245.474821587544,
    "mae": 26658.96854508142,
    "r2": 0.37770369938464365
}

===== Fold 4/10 =====
Métricas do fold:
{
    "fold": 4,
    "n_train": 20510,
    "n_val": 2279,
    "mse": 1017799655.5

/home/marlon/data-related-n3/.venv/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/marlon/data-related-n3/.venv/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/marlon/data-related-n3/.venv/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(



===== Treinando modelo final em full-data =====

===== Métricas do Modelo Final (full-data) =====
{
    "mse": 1036811868.683169,
    "rmse": 32199.5631753471,
    "mae": 26551.29983664157,
    "r2": 0.36521896252685093
}
Modelo final salvo em: ../artifacts/mlp/final_model.pkl
Resumo das métricas de K-Fold salvo em: ../artifacts/mlp/kfold_summary.json
Métricas do modelo final salvas em: ../artifacts/mlp/final_model_metrics.json
